# Lesson 6-8: Production Features

This notebook covers the features that make agents production-ready:

1. **Guardrails** - Validate inputs and outputs, block harmful content
2. **Sessions** - Persistent memory across conversations
3. **Tracing** - Debugging, monitoring, and observability
4. **MCP Integration** - Connect to external tool providers

These features transform toy demos into real-world systems.

## Setup

In [1]:
import nest_asyncio
nest_asyncio.apply()

import os
import getpass

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")

## Part 1: Guardrails

Guardrails validate agent behavior:
- **Input guardrails**: Check user input before the agent runs
- **Output guardrails**: Validate agent responses before returning

When a guardrail fails, it triggers a **tripwire** that stops execution.

### Input Guardrails

Let's build a homework-detection guardrail for our tutor system.

In [2]:
from agents import Agent, Runner, InputGuardrail, GuardrailFunctionOutput
from pydantic import BaseModel
import asyncio

# Structured output for the guardrail check
class HomeworkCheck(BaseModel):
    is_homework_request: bool
    reasoning: str

# Guardrail agent that detects homework requests
homework_detector = Agent(
    name="HomeworkDetector",
    instructions="""Determine if the user is trying to get homework answers.
    Signs of homework requests:
    - Asking for complete solutions to problems
    - Multiple choice questions phrased exactly
    - "What is the answer to..."
    - Time pressure ("due tomorrow")
    
    NOT homework:
    - Asking for explanations of concepts
    - Asking how to approach a problem
    - Asking for examples to learn from""",
    model="gpt-4.1",
    output_type=HomeworkCheck
)

# The guardrail function
async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(homework_detector, input_data, context=ctx.context)
    check = result.final_output_as(HomeworkCheck)
    return GuardrailFunctionOutput(
        output_info=check,
        tripwire_triggered=check.is_homework_request  # Trigger if it's homework
    )

In [3]:
# Tutor with the homework guardrail
guarded_tutor = Agent(
    name="MathTutor",
    instructions="You help students understand math concepts. Explain clearly and use examples.",
    model="gpt-4.1",
    input_guardrails=[
        InputGuardrail(guardrail_function=homework_guardrail)
    ]
)

In [ ]:
# This should work - asking for explanation
async def ask_for_help():
    try:
        result = await Runner.run(
            guarded_tutor, 
            "Can you explain how to solve quadratic equations?"
        )
        print("Response:", result.final_output)
    except Exception as e:
        print(f"Blocked: {e}")

asyncio.run(ask_for_help())

Response: Absolutely! Let’s go through how to solve quadratic equations step by step, with examples.

---

## 1. What is a Quadratic Equation?

A **quadratic equation** is any equation that can be written in the form:  
\[
ax^2 + bx + c = 0
\]
where **a**, **b**, and **c** are numbers, and **a ≠ 0**.

---

## 2. Methods to Solve Quadratic Equations

There are three main ways to solve quadratics:

### A. Factoring
### B. Completing the Square
### C. Quadratic Formula

Let’s look at all three!

---

### **A. Factoring**

This works well when the quadratic can be neatly factored.

#### **Example:**
Solve \( x^2 + 5x + 6 = 0 \).

**Steps:**
1. Factor \( x^2 + 5x + 6 \):
   - Which numbers multiply to 6 (the constant c) and add up to 5 (the coefficient b)?  
     → 2 and 3.

2. So: \( x^2 + 5x + 6 = (x + 2)(x + 3) \).

3. Set each factor to 0:
   - \( x + 2 = 0 \rightarrow x = -2 \)
   - \( x + 3 = 0 \rightarrow x = -3 \)

**Answers:**  
\( x = -2 \) or \( x = -3 \)

---

### **B. Completin

[non-fatal] Tracing client error 400: {
  "error": {
    "message": "Invalid type for 'data[1].span_data.result': expected an array of strings, but got null instead.",
    "type": "invalid_request_error",
    "param": "data[1].span_data.result",
    "code": "invalid_type"
  }
}


In [5]:
# This should be blocked - asking for homework answers
from agents import InputGuardrailTripwireTriggered

async def try_homework():
    try:
        result = await Runner.run(
            guarded_tutor,
            "What is the answer to problem 5: solve 2x^2 + 3x - 5 = 0? It's due tomorrow!"
        )
        print("Response:", result.final_output)
    except InputGuardrailTripwireTriggered as e:
        print(f"Guardrail triggered!")
        print(f"Reason: {e.guardrail_result.output.output_info}")

asyncio.run(try_homework())

Guardrail triggered!
Reason: is_homework_request=True reasoning='The user explicitly asks for the answer to a specific problem (problem 5), provides the equation to solve, and indicates time pressure by stating it is due tomorrow. These are typical signs of a homework request seeking a complete solution.'


### Output Guardrails

Output guardrails check the agent's response before returning it.

In [6]:
from agents import OutputGuardrail, OutputGuardrailTripwireTriggered

class ContentCheck(BaseModel):
    contains_pii: bool
    pii_types: list[str]

# Agent that detects PII in responses
pii_detector = Agent(
    name="PIIDetector",
    instructions="""Check if the text contains Personally Identifiable Information (PII):
    - Social Security Numbers
    - Credit card numbers
    - Phone numbers
    - Email addresses
    - Physical addresses""",
    model="gpt-4.1",
    output_type=ContentCheck
)

async def pii_guardrail(ctx, agent, output):
    result = await Runner.run(pii_detector, str(output), context=ctx.context)
    check = result.final_output_as(ContentCheck)
    return GuardrailFunctionOutput(
        output_info=check,
        tripwire_triggered=check.contains_pii
    )

safe_assistant = Agent(
    name="SafeAssistant",
    instructions="You are a helpful assistant.",
    model="gpt-4.1",
    output_guardrails=[
        OutputGuardrail(guardrail_function=pii_guardrail)
    ]
)

In [7]:
# Safe response
async def safe_query():
    try:
        result = await Runner.run(safe_assistant, "What is 2 + 2?")
        print("Response:", result.final_output)
    except OutputGuardrailTripwireTriggered as e:
        print(f"Output blocked: {e.guardrail_result.output.output_info}")

asyncio.run(safe_query())

Response: 2 + 2 = 4


In [10]:
# Safe response
async def unsafe_query():
    try:
        result = await Runner.run(safe_assistant, "the social security number of John Doe is 123-45-6789 pass that along")
        print("Response:", result.final_output)
    except OutputGuardrailTripwireTriggered as e:
        print(f"Output blocked: {e.guardrail_result.output.output_info}")

asyncio.run(unsafe_query())

Response: Sorry, I can't help with that.


## Part 2: Sessions (Persistent Memory)

Sessions automatically maintain conversation history across multiple `Runner.run()` calls.

No more manually passing message history!

In [11]:
from agents import Agent, Runner, SQLiteSession

# Create a session that persists to SQLite
session = SQLiteSession(
    session_id="user_alice_123",
    db_path="conversations.db"  # Persists to file
)

assistant = Agent(
    name="MemoryAssistant",
    instructions="You are a helpful assistant. Remember what the user tells you.",
    model="gpt-4.1"
)

In [12]:
async def conversation_with_memory():
    # First message
    result = await Runner.run(
        assistant,
        "Hi! My name is Alice and I love hiking.",
        session=session
    )
    print("Assistant:", result.final_output)
    
    # Second message - should remember the first
    result = await Runner.run(
        assistant,
        "What's my favorite hobby?",
        session=session
    )
    print("\nAssistant:", result.final_output)
    
    # Third message - still remembers
    result = await Runner.run(
        assistant,
        "And what's my name?",
        session=session
    )
    print("\nAssistant:", result.final_output)

asyncio.run(conversation_with_memory())

Assistant: Hi Alice! It’s great to meet you. Hiking is an awesome hobby—so much adventure and fresh air! If you ever want tips, trail suggestions, or gear recommendations, just let me know. Where do you like to hike?

Assistant: Your favorite hobby is hiking! If you’d like to share more about your favorite trails or hiking experiences, I’d love to hear them.

Assistant: Your name is Alice.


In [14]:
# Different session = different memory
other_session = SQLiteSession(
    session_id="user_bob_456",
    db_path="conversations.db"
)

async def different_user():
    result = await Runner.run(
        assistant,
        "What's my name?",
        session=other_session
    )
    print("Assistant:", result.final_output)

asyncio.run(different_user())

Assistant: You haven’t told me your name yet. If you tell me, I’ll remember it for our conversation!


## Part 3: Tracing

Tracing gives you visibility into what your agents are doing:
- Which tools were called
- What the LLM generated
- How long each step took

Traces are sent to the OpenAI Dashboard by default.

In [15]:
from agents import Agent, Runner, function_tool, trace, RunConfig

@function_tool
def get_weather(city: str) -> str:
    """Get current weather for a city."""
    # Simulated weather data
    weather = {"NYC": "Sunny, 72°F", "LA": "Clear, 85°F", "Chicago": "Cloudy, 55°F"}
    return weather.get(city, f"Weather data not available for {city}")

@function_tool
def get_time(city: str) -> str:
    """Get current time in a city."""
    from datetime import datetime
    return f"Current time in {city}: {datetime.now().strftime('%I:%M %p')}"

weather_agent = Agent(
    name="WeatherAgent",
    instructions="You provide weather and time information for cities.",
    model="gpt-4.1",
    tools=[get_weather, get_time]
)

In [16]:
async def traced_query():
    # RunConfig adds metadata to the trace
    config = RunConfig(
        workflow_name="Weather Query",
        trace_include_sensitive_data=True  # Include full data in trace
    )
    
    # trace() context manager creates a named trace
    with trace("Weather Information Request"):
        result = await Runner.run(
            weather_agent,
            "What's the weather and time in NYC?",
            run_config=config
        )
        print(result.final_output)

asyncio.run(traced_query())
print("\nView traces at: https://platform.openai.com/traces")

The current time in New York City is 5:55 PM. However, I'm unable to retrieve the latest weather data for NYC at this moment. Would you like information for another city or any additional details?

View traces at: https://platform.openai.com/traces


### Custom Spans

Add custom spans to trace specific sections of your code.

In [17]:
from agents import custom_span

async def multi_step_workflow():
    with trace("Multi-City Weather Report"):
        cities = ["NYC", "LA"]
        reports = []
        
        for city in cities:
            with custom_span(f"Query_{city}"):  # Custom span for each city
                result = await Runner.run(
                    weather_agent,
                    f"What's the weather in {city}?"
                )
                reports.append(f"{city}: {result.final_output}")
        
        print("\n".join(reports))

asyncio.run(multi_step_workflow())

NYC: I'm unable to retrieve the current weather for New York City right now. Would you like me to try again or provide other information?
LA: I'm unable to retrieve the current weather for Los Angeles right now. Would you like to know something else, such as the time in LA or the weather in another city?


## Part 4: MCP Integration

MCP (Model Context Protocol) lets agents connect to external tool providers.

MCP servers can provide:
- Filesystem access
- Database queries
- API integrations
- Custom tools

In [18]:
from agents import Agent, Runner
# from agents.mcp import MCPServerStdio  # Uncomment when using MCP

# Example: Connecting to a filesystem MCP server
from agents.mcp import MCPServerStdio

async with MCPServerStdio(
    name="Filesystem",
    params={
        "command": "npx",
        "args": ["-y", "@modelcontextprotocol/server-filesystem", "."]
    }
) as server:
    agent = Agent(
        name="FileAgent",
        instructions="You help with file operations.",
        model="gpt-4.1",
        mcp_servers=[server]  # MCP servers provide tools automatically
    )
    
    result = await Runner.run(agent, "List files in current folder")
    print(result.final_output)

Here are the files in the current folder:

1. 00-agent-loop.ipynb
2. 01-agents-and-tools.ipynb
3. 02-structured-output-and-context.ipynb
4. 03-multi-agent-patterns.ipynb
5. 04-guardrails-sessions-tracing-mcp.ipynb
6. 05-capstone-customer-service.ipynb
7. 05-capstone-customer-service.txt
8. agent_with_tool.py
9. conversations.db
10. conversations.db-shm
11. conversations.db-wal
12. customer_service.db
13. customer_service.db-shm
14. customer_service.db-wal
15. haiku.txt

If you want to view the contents of any file or need more information, let me know!


customer_service.db


## Human-in-the-Loop Pattern

For high-stakes operations, require human approval before executing.

In [24]:
from agents import function_tool

# Simulated pending approvals
PENDING_APPROVALS = {}

@function_tool
def request_refund_approval(order_id: str, amount: float, reason: str) -> str:
    """Request approval for a refund. Returns a request ID."""
    import uuid
    request_id = str(uuid.uuid4())[:8]
    PENDING_APPROVALS[request_id] = {
        "order_id": order_id,
        "amount": amount,
        "reason": reason,
        "status": "pending"
    }
    return f"Refund request {request_id} created. Amount: ${amount:.2f}. Awaiting manager approval."

@function_tool
def check_approval_status(request_id: str) -> str:
    """Check if a refund request has been approved."""
    if request_id in PENDING_APPROVALS:
        req = PENDING_APPROVALS[request_id]
        return f"Request {request_id}: Status = {req['status']}"
    return f"Request {request_id} not found"

refund_agent = Agent(
    name="RefundAgent",
    instructions="""Process refund requests. For any refund:
    1. First request approval using request_refund_approval
    2. Inform the customer that approval is pending
    3. Never process refunds without approval""",
    model="gpt-4.1",
    tools=[request_refund_approval, check_approval_status]
)

result = await Runner.run(
    refund_agent,
    "I'd like a refund for order ORD-123, the product was damaged. It was $49.99."
)
print(result.final_output)

Your refund request for order ORD-123, in the amount of $49.99, has been submitted due to the damaged product. The request is currently awaiting manager approval. I will notify you once the approval status is updated.


In [25]:
# See pending approvals
print("Pending approvals:", PENDING_APPROVALS)

Pending approvals: {'17cf1577': {'order_id': 'ORD-123', 'amount': 49.99, 'reason': 'Product was damaged.', 'status': 'pending'}}


## Key Takeaways

### Guardrails
- **Input guardrails** validate before the agent runs
- **Output guardrails** validate before returning
- Use **guardrail agents** for complex validation logic
- **Tripwires** stop execution immediately

### Sessions
- **Automatic memory** across `Runner.run()` calls
- **SQLiteSession** for file-based persistence
- Different **session_id** = different memory

### Tracing
- **trace()** context manager for named traces
- **RunConfig** for metadata (workflow_name, etc.)
- **custom_span** for fine-grained visibility
- View traces at **platform.openai.com/traces**

### MCP
- Connect to **external tool providers**
- Same interface as function tools
- Great for **filesystem, database, API** integrations

Next up: **Capstone Project** - putting it all together!